In [2]:
# importing required libraries
import numpy as np
import pandas as pd

import datetime as dt
import yfinance as yf
import pandas_datareader.data as web

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from scipy.optimize import minimize

In [13]:
#!pip install yfinance --upgrade --no-cache-dir

In [4]:
# random selection

# customize seed for different stocks
np.random.seed(42)

# list of all potential stocks (can be customized)
ticker_list = ['AAPL','AXP', 'BAC', 'C', 'CSCO', 'GS', 'IBM', 'INTC', 'JPM', 'MSFT', 'NVDA', 'CRM', 'QCOM', 'NOW', 'ORCL', 'AVGO', 'TXN', 'GDDY',
               'WIX', 'TSM', 'TSLA', 'SNOW', 'HUBS', 'DOCU', 'MS']

# selecting 15 random stocks
stock_list = np.random.choice(ticker_list,15,replace=False)
print(f'These are the fifteen stocks assigned to you: {" ".join(stock_list)}')

# sorting the list from A-Z
stock_list = list(np.sort(stock_list))

These are the fifteen stocks assigned to you: JPM TXN AAPL DOCU CRM MSFT NOW AXP HUBS GS BAC QCOM AVGO C CSCO


In [14]:
# finding optimal historical portfolio since 2000 to today, using 15 stocks. 
# Assume no short-selling constraints

# defining start and end date
start = dt.datetime(2000, 1, 1)
end = dt.datetime(2024, 12, 31)

# getting returns of the stocks
# calculates returns using adjusted close price
# ensures that we have returns of only those dates where all companies have available information
returns = yf.download(stock_list, start-pd.offsets.BDay(1), end+pd.offsets.BDay(1), auto_adjust=False)['Adj Close'].pct_change().dropna()

# gets all returns Close, high, low, open, volume
#returns = yf.download(stock_list, start, end)

returns.head()

[*********************100%***********************]  15 of 15 completed


Ticker,AAPL,AVGO,AXP,BAC,C,CRM,CSCO,DOCU,GS,HUBS,JPM,MSFT,NOW,QCOM,TXN
Date,,,,,,,,,,,,,,,
2018-04-30,0.018112,-0.014434,-0.008633,-0.007628,-0.010436,0.006405,-0.009394,-0.027687,-0.006130,-0.018081,-0.005667,-0.024003,0.018140,-0.001956,-0.010536
2018-05-01,0.023236,0.004185,-0.001721,0.001003,-0.000293,0.017522,0.012192,0.021227,-0.006965,-0.005666,0.000000,0.015826,-0.004334,-0.003725,0.019423
2018-05-02,0.044175,-0.007119,-0.009637,-0.012354,-0.003809,-0.002356,-0.021637,-0.003295,-0.011071,0.014245,-0.007906,-0.015684,0.000484,-0.011019,-0.004256
2018-05-03,0.001812,-0.015257,-0.005019,-0.012846,-0.000736,0.012946,0.013224,0.067141,-0.002734,0.010300,-0.006301,0.005989,0.019517,0.000995,0.006216
2018-05-04,0.039233,0.023396,0.012456,0.003424,0.004733,0.005707,0.019352,-0.020734,0.006555,0.019926,0.011097,0.011587,0.014757,0.043331,0.016605


In [15]:
# getting monthly returns
returns_mon = returns.resample(rule = 'ME').apply(lambda x: x.add(1).prod().sub(1))
returns_mon

Ticker,AAPL,AVGO,AXP,BAC,C,CRM,CSCO,DOCU,GS,HUBS,JPM,MSFT,NOW,QCOM,TXN
Date,,,,,,,,,,,,,,,
2018-04-30,0.018112,-0.014434,-0.008633,-0.007628,-0.010436,0.006405,-0.009394,-0.027687,-0.006130,-0.018081,-0.005667,-0.024003,0.018140,-0.001956,-0.010536
2018-05-31,0.135124,0.098728,-0.004557,-0.025446,-0.018520,0.068931,-0.035674,0.289412,-0.048887,0.144476,-0.016271,0.061467,0.069038,0.151289,0.109966
2018-06-30,-0.009418,-0.024502,-0.003052,-0.029270,0.003449,0.054666,0.007492,0.063040,-0.023508,0.034654,-0.026259,-0.002327,-0.028940,-0.034412,-0.014833
2018-07-31,0.027983,-0.086012,0.019156,0.095424,0.074268,0.005498,-0.009537,0.017941,0.076438,-0.010367,0.109162,0.075753,0.020235,0.142017,0.015295
2018-08-31,0.200422,-0.012355,0.064912,0.001619,-0.002795,0.113234,0.129581,0.158442,0.004917,0.157937,-0.003219,0.062993,0.115935,0.072086,0.009702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,0.032353,0.013319,0.022170,0.010915,-0.025235,-0.022798,0.043137,0.067231,0.008327,0.004104,0.056391,-0.001095,0.049866,-0.031224,0.051666
2024-09-30,0.017467,0.062937,0.048521,-0.019909,-0.000639,0.083985,0.053027,0.048640,-0.029672,0.065181,-0.062011,0.031548,0.046070,-0.024936,-0.036251
2024-10-31,-0.030429,-0.015826,-0.001528,0.053931,0.025080,0.064521,0.036989,0.117410,0.045808,0.043623,0.058901,-0.055659,0.043158,-0.042811,-0.010033


In [16]:
# S&P 500
sp500 = yf.Ticker("^GSPC")
sp500_data = sp500.history(period="25y")
sp500_data['Returns'] = sp500_data['Close'].pct_change()
sp500_data = sp500_data.drop(columns=["Open", "High", "Low", "Close", "Volume", "Dividends", "Stock Splits"])
sp500_data

,Returns
Date,
2000-03-20 00:00:00-05:00,NaN
2000-03-21 00:00:00-05:00,0.025566
2000-03-22 00:00:00-05:00,0.004532
2000-03-23 00:00:00-05:00,0.017799
2000-03-24 00:00:00-05:00,0.000072
...,...
2025-03-13 00:00:00-04:00,-0.013891
2025-03-14 00:00:00-04:00,0.021266
2025-03-17 00:00:00-04:00,0.006416


In [17]:
sp500_mon = sp500_data.resample(rule = 'ME').apply(lambda x: x.add(1).prod().sub(1))
sp500_mon

,Returns
Date,
2000-03-31 00:00:00-05:00,0.028799
2000-04-30 00:00:00-04:00,-0.030796
2000-05-31 00:00:00-04:00,-0.021915
2000-06-30 00:00:00-04:00,0.023934
2000-07-31 00:00:00-04:00,-0.016341
...,...
2024-11-30 00:00:00-05:00,0.057301
2024-12-31 00:00:00-05:00,-0.024990
2025-01-31 00:00:00-05:00,0.027016


In [18]:
# combining S&P500 with stock list



In [19]:
# get monthly std dev (volatility)
# get monthly SPX returns (historical)

In [20]:
rf = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0][['RF']].div(100)

rf.index = rf.index.to_timestamp(how='end').normalize()
rf

C:\Users\rishi\AppData\Local\Temp\ipykernel_39668\2736375131.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rf = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0][['RF']].div(100)
C:\Users\rishi\AppData\Local\Temp\ipykernel_39668\2736375131.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rf = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0][['RF']].div(100)


,RF
Date,
2000-01-31,0.0041
2000-02-29,0.0043
2000-03-31,0.0047
2000-04-30,0.0046
2000-05-31,0.0050
...,...
2024-08-31,0.0048
2024-09-30,0.0040
2024-10-31,0.0039


In [21]:
# Align indices of returns_mon and rf['RF']
aligned_rf = rf.reindex(returns_mon.index, method='pad')

In [22]:
aligned_rf

,RF
Date,
2018-04-30,0.0014
2018-05-31,0.0014
2018-06-30,0.0014
2018-07-31,0.0016
2018-08-31,0.0016
...,...
2024-08-31,0.0048
2024-09-30,0.0040
2024-10-31,0.0039
